In [4]:
# Import the methods that help in loading data and dealing with datasets:
import warnings 
warnings.filterwarnings("ignore")
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [5]:
# Import the data, convert the data into floating-point numbers, and register them to a device:
# Provide the data points to work on:

x = [[1,2],[3,4],[5,6],[7,8]]
y = [[3],[7],[11],[15]]

# Convert the data into floating-point numbers:

X = torch.tensor(x).float()
Y = torch.tensor(y).float()

# Register data to the device – given that we are working on a GPU,specify that the device is 'cuda'. If you are working on a CPU, specify the device as 'cpu':

device = 'cuda' if torch.cuda.is_available() else 'cpu'
X = X.to(device)
Y = Y.to(device)

In [6]:
class MyDataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x).float()
        self.y = torch.tensor(y).float()
    def __len__(self):
        return len(self.x)
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]
ds = MyDataset(X, Y)

In [7]:
# Pass the dataset instance defined previously through DataLoader to fetch
# the batch_size number of data points from the original input and output
# tensor objects:

dl = DataLoader(ds, batch_size=2, shuffle=True)

In [8]:
class MyNeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_to_hidden_layer = nn.Linear(2,8)
        self.hidden_layer_activation = nn.ReLU()
        self.hidden_to_output_layer = nn.Linear(8,1)
    def forward(self, x):
        x = self.input_to_hidden_layer(x)
        x = self.hidden_layer_activation(x)
        x = self.hidden_to_output_layer(x)
        return x

In [10]:
mynet = MyNeuralNet().to(device)
loss_func = nn.MSELoss()
from torch.optim import SGD
opt = SGD(mynet.parameters(), lr = 0.001)

In [11]:
# loop through the batches of data points to minimize the loss value
import time
loss_history = []
start = time.time()
for _ in range(50):
    for data in dl:
        x, y = data
        opt.zero_grad()
        loss_value = loss_func(mynet(x),y)
        loss_value.backward()
        opt.step()
        loss_history.append(loss_value)
end = time.time()
print(end - start)

0.11293935775756836


## Predicting on New Data Points

In [12]:
# Create the data points to test our model on:

val_x = [[10,11]]

In [13]:
# Convert the new data points into a tensor float object and register to the device:

val_x = torch.tensor(val_x).float().to(device)

In [14]:
# Pass the tensor object through the trained neural network – mynet – as if it were a Python function. This is the same as performing a forward
# propagation through the model that was built:

mynet(val_x)

tensor([[21.4901]], grad_fn=<AddmmBackward0>)